In [1]:
import datetime
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
import ribasim.nodes

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import *
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
waterschap = "Rijnland"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = -2  # default LevelBoundary level

## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_3"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed
)
processor.run()

Swapped edge direction between Node A: 297 and Node B: 642
Swapped edge direction between Node A: 642 and Node B: 15
Swapped edge direction between Node A: 307 and Node B: 477
Swapped edge direction between Node A: 477 and Node B: 2
Swapped edge direction between Node A: 2 and Node B: 681
Swapped edge direction between Node A: 681 and Node B: 311
Swapped edge direction between Node A: 232 and Node B: 600
Swapped edge direction between Node A: 600 and Node B: 65
Swapped edge direction between Node A: 148 and Node B: 1248
Swapped edge direction between Node A: 1248 and Node B: 22
Swapped edge direction between Node A: 11 and Node B: 712
Swapped edge direction between Node A: 712 and Node B: 15
Swapped edge direction between Node A: 11 and Node B: 661
Swapped edge direction between Node A: 661 and Node B: 15
Swapped edge direction between Node A: 27 and Node B: 897
Swapped edge direction between Node A: 897 and Node B: 15
Swapped edge direction between Node A: 33 and Node B: 633
Swapped e

#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

In [5]:
ribasim_model.pump.static.df.loc[ribasim_model.pump.static.df.node_id == 1117]

,node_id,active,flow_rate,min_flow_rate,max_flow_rate,min_upstream_level,max_downstream_level,control_state,meta_func_afvoer,meta_func_aanvoer,meta_func_circulatie,meta_type_verbinding,meta_node_id
fid,,,,,,,,,,,,,
223,1117,<NA>,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,1,0,0,Inlaat,<NA>


# Parameterization

## Nodes

### Basin (characteristics)

In [6]:
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


In [7]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [8]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [9]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

In [10]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

## Implement standard profile and a storage basin

In [11]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [12]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Basin (forcing)

In [13]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10*2),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [14]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [15]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

### Add Outlet

In [16]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [17]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [18]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [19]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [20]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


In [21]:
# #overwrite some control
# ribasim_model.outlet.static.df.max_downstream_level = np.nan
# ribasim_model.pump.static.df.loc[ribasim_model.pump.static.df.meta_func_afvoer, 'max_downstream_level'] = np.nan

### Manning Resistance

In [22]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [23]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 10
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [24]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

In [25]:
# ribasim_model.edge.df["fid"] = ribasim_model.edge.df.index.copy()

# Set numerical settings

In [26]:
ribasim_model.use_validation = True

In [27]:
# Write model output
# ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2025, 1, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

## Iterate over tabulated rating curves

In [28]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [29]:
controle_output = Control(work_dir=work_dir)
test = controle_output.run_all()

In [30]:
# try:
#     ribasim_param.iterate_TRC(
#         ribasim_param=ribasim_param,
#         allowed_tolerance=0.02,
#         max_iter=1,
#         expected_difference=0.1,
#         max_adjustment=0.25,
#         cmd=["ribasim", path_ribasim_toml],
#         output_dir=output_dir,
#         path_ribasim_toml=path_ribasim_toml,
#     )

# except Exception:
#     logging.error("The model was not able to run. Log file:")
#     log_file_path = os.path.join(output_dir, "ribasim.log")  # Update with the correct path to your log file
#     try:
#         with open(log_file_path) as log_file:
#             log_content = log_file.read()
#             print(log_content)
#     except Exception as log_exception:
#         logging.error(f"Could not read the log file: {log_exception}")

# Write model

In [31]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)

../../../../../Ribasim_networks/Waterschappen/Rijnland/modellen/Rijnland_parametrized/results
The model of waterboard Rijnland has been uploaded to the goodcloud in the directory of boezemmodel!


## Open Output

### Add discrete control nodes

In [32]:
# Add discrete control nodes and control edges
# ribasim_param.add_discrete_control_nodes(ribasim_model)

